<a href="https://colab.research.google.com/github/serotoninpm/do_it_nlp/blob/main/2%EC%9E%A5_%ED%86%A0%ED%81%B0%ED%99%94%ED%95%98%EA%B8%B0_tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패키지 설치
pip 명령어로 의존성 있는 패키지를 설치합니다.



In [1]:
!pip install ratsnlp

     |████████████████████████████████| 42 kB 286 kB/s 
     |████████████████████████████████| 57 kB 3.6 MB/s 
     |████████████████████████████████| 806 kB 20.1 MB/s 
     |████████████████████████████████| 2.8 MB 25.5 MB/s 
     |████████████████████████████████| 134 kB 8.5 MB/s 
     |████████████████████████████████| 829 kB 13.7 MB/s 
     |████████████████████████████████| 397 kB 41.2 MB/s 
     |████████████████████████████████| 636 kB 55.2 MB/s 
     |████████████████████████████████| 895 kB 48.1 MB/s 
     |████████████████████████████████| 3.3 MB 24.6 MB/s 
     |████████████████████████████████| 67 kB 4.9 MB/s 
     |████████████████████████████████| 1.1 MB 44.5 MB/s 
     |████████████████████████████████| 96 kB 4.1 MB/s 
     |████████████████████████████████| 271 kB 54.2 MB/s 
     |████████████████████████████████| 144 kB 72.4 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha

# 구글 드라이브 연동하기

지난 튜토리얼에서 구축한 어휘 집합을 저장해 둔 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


## GPT 입력값 만들기

GPT 모델 입력값을 만들려면 Byte-level Byte Pair Encoding 어휘집합 구축 결과(`vocab.json`, `merges.txt`)가 자신의 구글 드라이브 경로(`/gdrive/My Drive/nlpbook/wordpiece`)에 있어야 합니다. 다음을 수행해 이미 만들어 놓은 BBPE 어휘집합을 포함한 GPT 토크나이저를 `tokenizer_gpt`라는 변수로 선언합니다.

In [3]:
from transformers import GPT2Tokenizer
tokenizer_gpt = GPT2Tokenizer.from_pretrained("/gdrive/MyDrive/Colab_Notebooks/do_it_nlp/bbpe")
tokenizer_gpt.pad_token = "[PAD]"

file /gdrive/MyDrive/Colab_Notebooks/do_it_nlp/bbpe/config.json not found


예시 문장 세 개를 각각 토큰화해보겠습니다.

In [4]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_gpt.tokenize(sentence) for sentence in sentences]

토큰화 결과를 확인합니다.

In [5]:
tokenized_sentences

[['ìķĦ', 'ĠëįĶë¹Ļ', '..', 'Ġì§Ħì§ľ', 'Ġì§ľì¦ĿëĤĺ', 'ëĦ¤ìļĶ', 'Ġëª©ìĨĮë¦¬'],
 ['íĿł',
  '...',
  'íı¬ìĬ¤íĦ°',
  'ë³´ê³ł',
  'Ġì´ĪëĶ©',
  'ìĺģíĻĶ',
  'ì¤Ħ',
  '....',
  'ìĺ¤ë²Ħ',
  'ìĹ°ê¸°',
  'ì¡°ì°¨',
  'Ġê°Ģë³į',
  'ì§Ģ',
  'ĠìķĬ',
  'êµ¬ëĤĺ'],
 ['ë³Ħë£¨', 'Ġìĺ', 'Ģëĭ¤', '..']]

이번 배치의 크기가 3이라고 가정하고 이번 배치의 입력값을 만들어 보겠습니다.

In [6]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
batch_inputs = tokenizer_gpt(
    sentences,
    padding="max_length",
    max_length=12,
    truncation=True,
)

`batch_inputs`의 내용을 확인해보겠습니다.

In [7]:
batch_inputs.keys()

dict_keys(['input_ids', 'attention_mask'])

In [8]:
batch_inputs['input_ids']

[[334, 2338, 263, 581, 4055, 464, 3808, 0, 0, 0, 0, 0],
 [3693, 336, 2876, 758, 2883, 356, 806, 422, 9875, 875, 2960, 7292],
 [4957, 451, 3653, 263, 0, 0, 0, 0, 0, 0, 0, 0]]

In [9]:
batch_inputs['attention_mask']

[[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

In [20]:
sentences = [
    "가 나다라",
    "가나 다라",
    "가나다 라",
    "   가",
    " 가 ",
    "가   "
]
batch_inputs = tokenizer_gpt(
    sentences,
    padding="max_length",
    max_length=12,
    truncation=True,
)

In [21]:
batch_inputs['input_ids']

[[286, 400, 265, 326, 0, 0, 0, 0, 0, 0, 0, 0],
 [286, 315, 409, 326, 0, 0, 0, 0, 0, 0, 0, 0],
 [286, 4998, 1537, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [221, 221, 482, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [482, 221, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [286, 221, 221, 221, 0, 0, 0, 0, 0, 0, 0, 0]]

## BERT 입력값 만들기

BERT 모델 입력값을 만들려면 자신의 구글 드라이브 경로(`/gdrive/My Drive/nlpbook/wordpiece`)에 워드피스 어휘집합 구축 결과(`vocab.txt`)가 있어야 합니다. 다음을 수행해 위에서 이미 만들어 놓은 워드피스 어휘집합을 포함한 BERT 토크나이저를 `tokenizer_bert`라는 변수로 선언합니다.

In [38]:
from transformers import BertTokenizer
tokenizer_bert = BertTokenizer.from_pretrained(
    "/gdrive/MyDrive/Colab_Notebooks/do_it_nlp/wordpiece/vocab.txt", 
    do_lower_case=False
)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1643: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  FutureWarning,


예시 문장 세 개를 각각 토큰화해보겠습니다.

In [39]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_bert.tokenize(sentence) for sentence in sentences]

토큰화 결과를 확인합니다.

In [40]:
tokenized_sentences

[['아', '더빙', '.', '.', '진짜', '짜증나', '##네요', '목소리'],
 ['흠',
  '.',
  '.',
  '.',
  '포스터',
  '##보고',
  '초딩',
  '##영화',
  '##줄',
  '.',
  '.',
  '.',
  '.',
  '오버',
  '##연기',
  '##조차',
  '가볍',
  '##지',
  '않',
  '##구나'],
 ['별루', '였다', '.', '.']]

이번 배치의 크기가 3이라고 가정하고 이번 배치의 입력값을 만들어 보겠습니다.

In [41]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
batch_inputs = tokenizer_bert(
    sentences,
    padding="max_length",
    max_length=12,
    truncation=True,
)

`batch_inputs`의 내용을 확인해보겠습니다.

In [42]:
batch_inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [43]:
batch_inputs['input_ids']

[[2, 621, 2631, 16, 16, 1993, 3678, 1990, 3323, 3, 0, 0],
 [2, 997, 16, 16, 16, 2609, 2045, 2796, 1981, 1467, 16, 3],
 [2, 3274, 9508, 16, 16, 3, 0, 0, 0, 0, 0, 0]]

In [44]:
batch_inputs['attention_mask']

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]

In [45]:
batch_inputs['token_type_ids']

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]